In [ ]:
import sys
from datetime import datetime, timedelta
import pandas as pd
import os
sys.path.append('../src/')
from Biologging_Toolkit.processing.Inertial import Inertial
from Biologging_Toolkit.applications.Jerk import Jerk
from Biologging_Toolkit.plot.spectrogram import *

#### Enter deployment ID and path

Finalized dataset will be saved or loaded from path

In [ ]:
depid = 'ml18_294b'
path = 'D:/individus_brut/'
sens_path = os.path.join(path, 'CTD', depid, depid+'sens5.nc')
raw_path = os.path.join(path, 'individus', depid, 'raw')

#### Run cells to get PCAs

Time can be constructed from extisting time array or by entering the path to sens5 dataset.

In [ ]:
aux = pd.read_csv(os.path.join(path, 'individus', depid, 'aux_data.csv'))  #Get existing dataframe with time array
inst = Jerk(depid, 
            path = os.path.join(path, 'individus',depid), 
            sens_path = sens_path, 
            raw_path = raw_path
           )


In [ ]:
inst.create_time(aux.time.to_numpy())
inst(overwrite = True)    #Calls low resolution jerk from sens5, then double checks with high resolution Acc data

#### Run cells to get posture and heading

In [ ]:
inst = Inertial(depid, 
            path = os.path.join(path, 'individus',depid), 
            inertial_path = sens_path, 
            )

In [ ]:
inst()

#### Plot values over spectrogram

In [ ]:
# Example parameters
debut = datetime(2018,10,25,3,14,30) + timedelta(seconds = 756)
fin = datetime(2018,10,25,3,14,30) + timedelta(seconds = 756+9*60)
debut = datetime.fromtimestamp(1540839669.5589542,  tz=timezone.utc)
fin = datetime.fromtimestamp(1540839906.5589733,  tz=timezone.utc)
freq_min = 250
freq_max = 400

In [ ]:
plot_spectrogram(inst, debut, fin, freq_min, freq_max, raw_path, nperseg = 8196)